In [1]:
import tensorflow as tf

print(tf.__version__)

2.4.0


In [2]:
import os

import matplotlib.pyplot as plt
import cv2
from google.colab import drive
# Accessing My Google Drive
drive.mount('/content/drive')
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense,GlobalAveragePooling2D,Convolution2D
from keras.applications import DenseNet121
from keras.layers import GaussianNoise
from keras.layers import Dropout
from keras.layers import BatchNormalization
from glob import glob
import numpy as np
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Activation
from keras.layers import concatenate
from keras.layers import UpSampling2D
from keras import Model

import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Mounted at /content/drive


In [3]:
training_path = 'drive/My Drive/ISIC_2019_Training_Input'
validation_path = 'drive/My Drive/groundtruth_val.csv'

Train_path =  'drive/My Drive/groundtruth_train.csv'
#train_data = Train_df['image']



train_data = pd.read_csv(Train_path, encoding='latin1', dtype={"image": str, "MEL": int, "NV": int, "BCC": int, "AK": int,"BKL": int, "": int, "DF": int, "VASC": int,"SCC": int, "UNK": int})
train_data['image'] =train_data['image']+'.jpg'
labels = np.argmax(np.array(train_data.iloc[:,1:10]), axis=1)
images = np.asarray(train_data.iloc[:, 0])
classes = list(train_data.columns.values[1:10])

print(classes)
print(images)
print(labels)


validation_data = pd.read_csv(validation_path, encoding='latin1', dtype={"image": str, "MEL": int, "NV": int, "BCC": int, "AK": int,"BKL": int, "": int, "DF": int, "VASC": int,"SCC": int, "UNK": int})
validation_data['image'] =validation_data['image']+'.jpg'
val_labels = np.argmax(np.array(validation_data.iloc[:,1:10]), axis=1)
val_images = np.asarray(validation_data.iloc[:, 0])
val_classes = list(validation_data.columns.values[1:10])
print(val_classes)
print(val_images)
print(val_labels)

['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
['ISIC_0059444.jpg' 'ISIC_0032074.jpg' 'ISIC_0059546.jpg' ...
 'ISIC_0014331_downsampled.jpg' 'ISIC_0032681.jpg' 'ISIC_0028228.jpg']
[0 1 2 ... 0 4 0]
['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
['ISIC_0026624.jpg' 'ISIC_0032373.jpg' 'ISIC_0012356_downsampled.jpg' ...
 'ISIC_0032760.jpg' 'ISIC_0033878.jpg' 'ISIC_0066391.jpg']
[1 4 0 ... 1 0 3]


In [7]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    flatten = Flatten()(conv10)
    dense1 = Dense(1024, activation='relu')(flatten)
    dense2 = Dense(512, activation='relu')(dense1)
    batch1 = BatchNormalization()(dense2)
    drop1 = Dropout(0.5)(batch1)

    preds = Dense(9,activation='softmax')(drop1)

    model = Model(inputs = inputs, outputs = preds)

    model.compile(optimizer = Adam(lr = 1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

y = unet()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_25[0][0]                  
____________________________________________________________________________________________

In [ ]:
callbacks_save = ModelCheckpoint('unetbest.h5',
                                 monitor='val_loss',
                                 mode='min',
                                 save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=1e-6, cooldown=1, verbose=1)


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.3, rotation_range=30,
                                   width_shift_range=0.3, height_shift_range=0.3,
                                   brightness_range=[0.2, 1.0], horizontal_flip=True,
                                   vertical_flip=True, fill_mode='nearest', zoom_range=0.4
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_dataframe(train_data,
                                                 directory= training_path,
                                                 x_col= 'image',
                                                 y_col= list(classes),
                                                 target_size =(256,256),
                                                 batch_size = 16,
                                                 class_mode ='raw',
                                                 validate_filenames=False)

test_set = test_datagen.flow_from_dataframe(validation_data,
                                                  directory= training_path,
                                                  x_col= 'image',
                                                  y_col= list(val_classes),
                                                  target_size =(256,256),
                                                  batch_size = 16,
                                                  class_mode ='raw',
                                                  validate_filenames=False )

y.fit_generator(training_set,
                         steps_per_epoch =len(training_set)/4,
                         epochs =15,
                         validation_data = test_set,
                         validation_steps =len(test_set)/4,
                         callbacks=[callbacks_save, reduce_lr])

y.save("unettrial.h5")

Found 22797 non-validated image filenames.
Found 2534 non-validated image filenames.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
357/356 [==============================] - ETA: -81s - loss: 1.6735 - accuracy: 0.4840